# Chapter 6. Performance analysis by server elimination

This Notebook computes and plots the numerical results for Chapter 6 of the Ph.D. thesis. Sections 1 and 2 of this Notebook define functions for computing the results, while Sections 3 and 4 plot them. For each section, we add a pointer to the corresponding section or subsection in the manuscript.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
folder = "data/"

## 1. Random customer assignment (Section 6.2)

### Homogeneous queue (Subsection 6.2.1)

$$
\psi = \prod_{s=r}^S \left( 1 - \rho_{|s} \right)
\quad \text{and} \quad
L = \sum_{s=r}^S \frac{ \rho_{|s} }{ 1 - \rho_{|s} },
$$
where
$$
\rho_{|s}
= \frac1{s \mu} \frac{ \binom{s}{r_n} }{ \binom{S}{r_n} } S \lambda
= \rho \frac{ \binom{s-1}{r_n-1} }{ \binom{S-1}{r_n-1} }
$$
is the load in the subsystem restricted to $s$ arbitrary servers.

We just need to compute
$$
    b_s = \binom{s-1}{r-1}
$$
for each $s = r,\ldots,S$.
We have $b_r = 1$ and, for each $s = r+1,\ldots,S$,
$$
    b_s
    = \frac{ (s-1)! }{ (r-1)! (s-r)! }
    = \frac{s-1}{s-r} \frac{ ((s-1)-1)! }{ (r-1)! ((s-1)-r)! }
    = \left( 1 + \frac{r-1}{s-r} \right) b_{s-1}.
$$
We use this last equality to compute recursively $b_s$ for each $s = r+1,\ldots,S$.

In [ ]:
def global_homogeneous(S=10, r=3, ρ=.5):
    binom = np.ones(S - r + 1, dtype=float)
    binom[1:] = np.cumprod(1 + (r-1) / np.arange(1, S - r + 1))
    binom /= binom[-1]
    ρs = ρ * binom
    return np.sum(ρs / (1 - ρs))

### Heterogeneous degrees (Subsection 6.2.2)

$$
\psi = \prod_{s=r}^S \left( 1 - \rho_{|s} \right)
\quad \text{and} \quad
L = \sum_{s=r}^S \frac{ \rho_{|s} }{ 1 - \rho_{|s} },
$$
where
$$
\rho_{|s} = \frac1{ s \mu }
\sum_{k=1}^K \frac{ \binom{s}{r_k} }{ \binom{S}{r_k} } S \lambda p_k
= \rho \sum_{k=1}^K \frac{ \binom{s-1}{r_k-1} }{ \binom{S-1}{r_k-1} } p_k
$$
is the load in the queue restricted to $s$ arbitrary servers.

Implementation: We let
$$
b_{k,s} = p_k \frac{ \binom{s}{r_k} }{ \binom{S}{r_k} },
$$
and we compute it by induction on $s$ for each $k$, using the formula:
$$
\binom{s}{r} = \left( 1 + \frac{r-1}{s-r} \right) \times \binom{s-1}{r}.
$$
Then we let $w_s = b_{1,s} + \ldots + b_{K,s}$ for each $s = 1,\ldots,S$.

We can express all the metrics as functions of these quantities:
\begin{align*}
\rho_s &= \rho \times w_s,
&\qquad \psi &= \prod_{s=1}^S (1 - \rho_{s}),
&\qquad L &= \sum_{s=1}^S \frac{ \rho_{|s} }{ 1 - \rho_{|s} }, \\
\rho_{k|s} &= \frac{ \rho \times b_{k,s} }{ 1 - \rho (w_s - b_{k,s}) },
&&
&\qquad L_k &= \sum_{s=r_k}^S \frac{ \rho_{k|s} }{ 1 - \rho_{k|s} }
\end{align*}

In [ ]:
def global_heterogeneous(S, r, p, ρ):
    K = len(r);
    b = np.zeros((K, S), dtype=float)
    for k in range(K):
        b[k, r[k] - 1] = 1.
        b[k, r[k]:] = np.cumprod(1 + (r[k] - 1) / np.arange(1, S - r[k] + 1))
        b[k, :] *= p[k] / b[k, -1]
    w = np.sum(b, axis=0)  # sum for all k = 1,...,K

    Ln = np.zeros(K)

    ρs = ρ * w
    ψ = np.prod(1. - ρs)
    L = np.sum(ρs / (1. - ρs))
    for k in range(K):
        ρns = ρ * b[k, :] / (1. - ρ * (w - b[k, :]))
        Ln[k] = np.sum(ρns / (1. - ρns))

    return ψ, L / S / ρ, Ln / S / ρ / p

## 2. Local assignment (Section 6.3)

### Random customer assignment (Subsection 6.3.3)

In a homogeneous, local, randomized load balancing system of parameters $ S $, $ r $ and $ \rho =\frac{\lambda}{\mu} $, the stability condition is $ \rho < 1 $. Defining $ \rho_{|s} = \frac{1-\frac{r-1}{s}}{1-\frac{r-1}{S}}\rho$, the probability that the system is empty is $\psi = \psi_{|1..S}$ , with
	$$
	\psi_{|1..s} = \left\{
	\begin{array}{l}
	1 \text{ if } s < r\text{,}\\
	(1-\rho_{|s})\frac{s}{\sum_{t=1}^{s}\frac{1}{\psi_{|1..t-1}\psi_{|1..s-t}}}
	\text{ otherwise.}
	\end{array}
	 \right.
	$$

In [ ]:
def line_ψ(S=10, r=3, ρ=.5):
    ψ = np.ones(S + 1)
    for s in range(r, S + 1):
        ρs = ρ * (1 - (r - 1) / s) / (1 - (r - 1) / S)
        ψ[s] = (1 - ρs) * s / np.sum([1 / ψ[t-1] / ψ[s-t] for t in range(1, s+1)])
    return ψ

For each $t \in 1..S-r+1$, the mean number of class-$t$ jobs in the system is $L_{t} = L_{t|1..S}$, with
\begin{equation}
L_{t|1..s} =
\frac{\frac{\rho_{|S}}{1-\frac{r-1}{S}}		
+\psi_{|1..s}\left(
\sum\limits_{u=1}^{t-1}\frac{L_{t-u|1..\ell-u}}{\psi_{|1..u-1}\psi_{|1..s-u}}
	+
\sum\limits_{u=t+r}^{s}\frac{L_{t|1..u-1}}{\psi_{|1..u-1}\psi_{|1..s-u}}
\right)
}{\ell(1-\rho_{|s})}\text{,}
\end{equation}

In [ ]:
def line_Li(S=10, r=3, ρ=.5):
    ψ = line_ψ(S, r, ρ)
    Lt = np.zeros([S - r + 2, S + 1])
    for t in range(1, S - r + 2):
        for s in range(t + r - 1, S + 1):
            ρs = ρ * (1 - (r - 1) / s) / (1 - (r - 1) / S)
            Lt[t, s] = ((ρ / (1 - (r - 1) / S)
                        + ψ[s] * (np.sum([Lt[t-u, s-u] / ψ[u-1] / ψ[s-u] for u in range(1, t)])
                                  + np.sum([Lt[t, u-1] / ψ[u-1] / ψ[s-u] for u in range(t+r, s+1)])))
                       / s / (1 - ρs))
    return Lt[1:,-1]

The total mean number of jobs in the system is $L = L_{|1..S}$, with
\begin{equation}
L_{|1..s} = \frac{
		\rho_{|s}+\frac{\psi_{|1..s}}{s}\sum_{t=1}^{s}\frac{L_{|1..t-1}+L_{|1..s-t}}{\psi_{|1..t-1}\psi_{|1..s-t}}
	}{1-\rho_{|s}
}\text{.}
\end{equation}

We check with a specific function that it matches the Lis.

In [ ]:
def line_L(S=10, r=3, ρ=.5):
    ψ = line_ψ(S, r, ρ)
    L = np.zeros(S + 1)
    for s in range(r, S + 1):
        ρs = ρ * (1 - (r - 1) / s) / (1 - (r - 1) / S)
        L[s] = (ρs + ψ[s] / s * np.sum([(L[t-1] + L[s-t]) / ψ[t-1] / ψ[s-t]
                                        for t in range(1, s + 1)])) / (1 - ρs)
    return L[-1]

### Ring queue (Subsection 6.3.4)

In [ ]:
def ring(S=10, r=3, ρ=.5):
    return ρ / (1 - ρ) + line_L(S - 1, r, (1 - (r - 1) / (S - 1)) * ρ)

## 3. Gain of differentiation (Subsection 6.4.1)

### Impact of load

In [ ]:
def diff_study(S=100, step=.001):
    ρρ = np.arange(step, 1, step)
    I = len(ρρ)
    r6 = np.zeros(I)
    r8 = np.zeros(I)
    r9 = np.zeros(I)
    r12 = np.zeros(I)
    rm = np.zeros(I)
    r6m = np.zeros(I)
    r12m = np.zeros(I)

    for i, ρ in enumerate(ρρ):
        _, r6[i], _ = global_heterogeneous(S, [4], [1], ρ)  # 6
        _, r8[i], _ = global_heterogeneous(S, [8], [1], ρ)
        _, r9[i], _ = global_heterogeneous(S, [9], [1], ρ)
        _, r12[i], _ = global_heterogeneous(S, [14], [1], ρ)  # 12
        _, rm[i], R = global_heterogeneous(S, [4, 14], [.5, .5], ρ)  # mix
        r6m[i] = R[0]
        r12m[i] = R[1]
    csv = pd.DataFrame({'rho': ρρ, 'd6': r6, 'd8': r8, 'd9': r9,
                        'd12': r12, 'd6m': r6m, 'd12m': r12m, 'dm': rm})
    fn = folder + "hetero1_" + str(S) + ".csv"
    csv.to_csv(fn, index=False)

In [ ]:
diff_study()

In [ ]:
csv = pd.read_csv(folder + "hetero1_100.csv")
plt.plot(csv['rho'].values, 1 / csv['d6'].values, label="r6")
plt.plot(csv['rho'].values, 1 / csv['d6m'].values, label="r6m")
plt.plot(csv['rho'].values, 1 / csv['d12m'].values, label="r12m")
plt.plot(csv['rho'].values, 1 / csv['d12'].values, label="r12")
plt.xlim([0, 1]); plt.ylim(ymin=0)
plt.legend(loc='best')
plt.show()

### Impact of population distribution

In [ ]:
S = 100
step = .001
ρl = .2
ρm = .9
ρh = .99

pp = np.arange(step, 1, step)
I = len(pp)
r6l = np.zeros(I)
r12l = np.zeros(I)
r6m = np.zeros(I)
r12m = np.zeros(I)
r6h = np.zeros(I)
r12h = np.zeros(I)

degrees = [6, 12]

for i, p in enumerate(pp):
    _, _, r = global_heterogeneous(S, degrees, [p, 1 - p], ρl)
    r6l[i] = r[0]
    r12l[i] = r[1]
    _, _, r = global_heterogeneous(S, degrees, [p, 1 - p], ρm)
    r6m[i] = r[0]
    r12m[i] = r[1]
    _, _, r = global_heterogeneous(S, degrees, [p, 1 - p], ρh)
    r6h[i] = r[0]
    r12h[i] = r[1]
csv = pd.DataFrame({'p': pp, 'r6l': r6l, 'r12l': r12l, 'r6m': r6m,
                    'r12m': r12m, 'r6h': r6h, 'r12h': r12h})
fn = folder + "hetero2.csv"
csv.to_csv(fn, index=False)

In [ ]:
csv = pd.read_csv(folder + "hetero2.csv")

plt.plot(csv['p'].values, 1 / csv['r6h'].values, label="r6h")
plt.plot(csv['p'].values, 1 / csv['r12h'].values, label="r12h")
plt.plot(csv['p'].values, 1 / csv['r6m'].values, label="r6m")
plt.plot(csv['p'].values, 1 / csv['r12m'].values, label="r12m")

plt.xlim([0, 1]); plt.ylim(ymin=0)
plt.legend(loc='best')
plt.show()

## 4. Impact of locality (Subsection 6.4.2)

### Cost of heterogeneity and locality

In [ ]:
S = 100
r = 10
ρ = .9999

Ls = line_Li(S, r, ρ)
ring_delay = ring(S, r, ρ)
range_a = line_L(S, r, ρ)
range_i = Ls * (S - r + 1)
R = global_homogeneous(S, r, ρ)

plt.plot([1, S - r + 1], 1. / (R * np.ones(2)), label="Global")
plt.plot([1, S - r + 1], 1. / (ring_delay * np.ones(2)), label="Ring")
plt.plot(range(1, S - r + 2), 1. / range_i, label="Line (classes)")
plt.plot([1, S - r + 1], 1. / (range_a * np.ones(2)), label="Line (mean)")

plt.xlim([1, S - r + 1])
plt.legend(loc='best')
plt.show()

In [ ]:
S = 100
r = 2
ρ = .999

Ls = line_Li(S, r, ρ)
ring_delay = ring(S, r, ρ)
range_a = line_L(S, r, ρ)
range_i = Ls * (S - r + 1)
R = global_homogeneous(S, r, ρ)

plt.plot([1, S - r + 1], 1. / (R * np.ones(2)), label="Global")
plt.plot([1, S - r + 1], 1. / (ring_delay * np.ones(2)), label="Ring")
plt.plot(range(1, S - r + 2), 1. / range_i, label="Line (classes)")
plt.plot([1, S - r + 1], 1. / (range_a * np.ones(2)), label="Line (mean)")

plt.xlim([0, S - r + 1]); plt.ylim(ymin=0)
plt.legend(loc='best')
plt.show()

In [ ]:
def load_study(S=100, r=10, ρ = .9):
    c = np.arange(1, S - r + 2)
    Li = line_Li(S, r, ρ) * (S - r + 1) / S / ρ
    L = line_L(S, r, ρ) / S / ρ * np.ones(S - r + 1)
    R = ring(S, r, ρ) / S / ρ * np.ones(S - r + 1)
    G = global_homogeneous(S, r, ρ) / S / ρ * np.ones(S - r + 1)
    csv = pd.DataFrame({'c': c, 'Li': Li, 'L': L, 'R': R, 'G': G})
    csv.to_csv(folder + "load_study_S_" + str(S) + "_r_" + str(r) + "_rho_" + str(ρ) + ".csv",
               index=False)

In [ ]:
def plot_local_study(S=100, r=10, ρ=.9):
    try:
        csv = pd.read_csv(folder + "load_study_S_" + str(S)
                          + "_r_" + str(r) + "_rho_" + str(ρ) + ".csv")
    except:
        load_study(S, r, ρ)
        csv = pd.read_csv(folder + "load_study_S_" + str(S)
                          + "_r_" + str(r) + "_rho_" + str(ρ) + ".csv")
    
    plt.plot(csv['c'].values, 1. / csv['G'].values, label="Global")
    plt.plot(csv['c'].values, 1. / csv['R'].values, label="Ring")
    plt.plot(csv['c'].values, 1. / csv['L'].values, label="Line (Mean)")
    plt.plot(csv['c'].values, 1. / csv['Li'].values, label="Line (Classes)")
    
    plt.xlim([1, S - r + 1])
    plt.legend(loc='best')
    plt.show()

In [ ]:
plot_local_study(ρ=.5)

### Impact of the parameters

#### Load $\rho$

In [ ]:
def load_study_rho(S=100, r=10, step=.001):
    ρρ = np.arange(step, 1, step)
    I = len(ρρ)
    L = np.zeros(I)
    R = np.zeros(I)
    G = np.zeros(I)
    
    for i, ρ in enumerate(ρρ):
        norm = S * ρ
        L[i] = line_L(S, r, ρ) / norm
        R[i] = ring(S, r, ρ) / norm
        G[i] = global_homogeneous(S, r, ρ) / norm
        
    csv = pd.DataFrame({'rho': ρρ, 'L': L, 'R': R, 'G': G})
    csv.to_csv(folder + "load_study_rho_S_" + str(S) + "_r_" + str(r) + ".csv",
               index=False)

In [ ]:
def plot_local_study_rho(S=100, r=10, step=.001):
    try:
        csv = pd.read_csv(folder + "load_study_rho_S_" + str(S) + "_r_" + str(r) + ".csv")
    except:
        load_study_rho(S, r, step)
        csv = pd.read_csv(folder + "load_study_rho_S_" + str(S) + "_r_" + str(r) + ".csv")
        
    plt.plot(csv['rho'].values, 1. / csv['G'].values, label="Global")
    plt.plot(csv['rho'].values, 1. / csv['R'].values, label="Ring")
    plt.plot(csv['rho'].values, 1. / csv['L'].values, label="Line")
    
    plt.xlim([0,1]); plt.ylim(0, 10)
    plt.legend(loc='best')
    plt.show()

In [ ]:
plot_local_study_rho(100, 10, step=.001)

#### Parallelism degree $r$

In [ ]:
def load_study_r(S=100, ρ=.9):
    rr = np.arange(1, S+1)
    I = len(rr)
    L = np.zeros(I)
    R = np.zeros(I)
    G = np.zeros(I)
    
    for i, r in enumerate(rr):
        norm = S * ρ
        L[i] = line_L(S, r, ρ) / norm
        R[i] = ring(S, r, ρ) / norm
        G[i] = global_homogeneous(S, r, ρ) / norm
        
    csv = pd.DataFrame({'r': rr, 'L': L, 'R': R, 'G': G})
    csv.to_csv(folder + "load_study_r_S_" + str(S) + "_rho_" + str(ρ) + ".csv",
               index=False)

In [ ]:
def plot_local_study_r(S=100, ρ=.9):
    try:
        csv = pd.read_csv(folder + "load_study_r_S_" + str(S) + "_rho_" + str(ρ) + ".csv")
    except:
        load_study_r(S, ρ)
        csv = pd.read_csv(folder + "load_study_r_S_" + str(S) + "_rho_" + str(ρ) + ".csv")
        
    plt.plot(csv['r'].values, 1. / csv['G'].values, label="Global")
    plt.plot(csv['r'].values, 1. / csv['R'].values, label="Ring")
    plt.plot(csv['r'].values, 1. / csv['L'].values, label="Line")
    
    plt.xlim([1,S]); plt.ylim(0, 10)
    plt.legend(loc='best')
    plt.show()

In [ ]:
plot_local_study_r(100, .9)

#### Number $S$ of servers

In [ ]:
def load_study_S(Smax=100, r=10, ρ=.9):
    SS = np.arange(r + 1, Smax + 1)
    I = len(SS)
    L = np.zeros(I)
    R = np.zeros(I)
    G = np.zeros(I)
    
    for i, S in enumerate(SS):
        norm = S * ρ
        L[i] = line_L(S, r, ρ) / norm
        R[i] = ring(S, r, ρ) / norm
        G[i] = global_homogeneous(S, r, ρ) / norm
        
    csv = pd.DataFrame({'S': SS, 'L': L, 'R': R, 'G': G})
    csv.to_csv(folder + "load_study_S_Smax_" + str(Smax)
               + "_r_" + str(r) + "_rho_" + str(ρ) + ".csv",
               index=False)

In [ ]:
def plot_local_study_S(Smax=100, r=10, ρ=.9):
    try:
        csv = pd.read_csv(folder + "load_study_S_Smax_" + str(Smax)
                          + "_r_" + str(r) + "_rho_" + str(ρ) + ".csv")
    except:
        load_study_S(Smax, r, ρ)
        csv = pd.read_csv(folder + "load_study_S_Smax_" + str(Smax)
                          + "_r_" + str(r) + "_rho_" + str(ρ) + ".csv")
        
    plt.plot(csv['S'].values, 1 / csv['G'].values, label="Global")
    plt.plot(csv['S'].values, 1 / csv['R'].values, label="Ring")
    plt.plot(csv['S'].values, 1 / csv['L'].values, label="Line")
    
    plt.xlim([r + 1, Smax]); plt.ylim(ymin=0)
    plt.legend(loc='best')
    plt.show()

In [ ]:
plot_local_study_S(300)